## Synthetic SAEM modelling 

We will do some synthetic modelling of a Drone-based SAEM survey to understand the fields, investigate the influence of several acquisition parameters, and analysing resolution.

In [1]:
# First import the SAEM class and the numpy module
import numpy as np
from saem import CSEMData

PRECORE


### Definition of the frequency axis

In [2]:
df = 0.5  # 0.5/1 for half/full octave
fMin, fMax = 32, 25000
f = np.round(2**np.arange(0, np.log2(fMax/fMin), df) * fMin)
print(f)

[   32.    45.    64.    91.   128.   181.   256.   362.   512.   724.
  1024.  1448.  2048.  2896.  4096.  5793.  8192. 11585. 16384. 23170.]


The higher frequencies follow the half-octave processing scheme. However, there are some lower frequencies that are normally not processed. We will get rid of them

In [3]:
f = np.hstack((f[3], f[5:-3], f[-2:]))
print(f)

[   91.   181.   256.   362.   512.   724.  1024.  1448.  2048.  2896.
  4096.  5793.  8192. 16384. 23170.]


### Acquisition parameters
Assume we have a South-North (y) trending transmitter and a receiver line along the x axis.
We initialize the CSEMdata class passing the Tx/Rx positions and frequencies.

In [4]:
x = np.arange(25, 1000, 50)  # receiver positions
txLen = 500  # length of the transmitter
self = CSEMData(f=f, rx=x, txPos=np.array([[0, 0], [-txLen/2, txLen/2]]), alt=30)
self.cmp = [1, 0, 1]
print(self)

ValueError: operands could not be broadcast together with shapes (19,) (0,) 

In [ ]:
self.showPos(); # show transmitter and receiver positions

### A synthetic model

Assume a two layer model of a freshwater aquifer above a clay layer situated at a depth of 100m. 
In the following we are only interested in the Bx and Bz component and set the cmp property.

In [ ]:
# self.simulate(rho=[100, 100], thk=[100])
self.simulate(rho=[300, 20], thk=[100])

In [ ]:
# We show the data along the line using the same amplitude and phase scale
self.showLineData(what="response", amphi=True, alim=[-3, 0], plim=[90, 180]);


The amplitudes of both components decrease with increasing distance and frequency. Whereas both components are similar close to the transmitter, the z component decreases way faster and is therefore more prone to noise. On the other hand, the phase shift is stronger for Bz.

In [ ]:
# We can also show real and imaginary part of the data. 
# As we don't see much in a linear scale, we use a symlog scale (decades over threshold).
self.showLineData(what="response");

The good conductor generates a significant imaginary (induction) part on the whole frequency axis, which however decreases rapidly with distance.

In [ ]:
# We now set the response to the data and add some random noise.
self.DATA = self.RESP.copy()
from numpy.random import randn
noiseLevel = 1e-3  # nV/A
sh = self.DATA.shape
noise = (randn(*sh) + randn(*sh) * 1j) * noiseLevel
self.DATA += noise
self.showLineData();

We easily see that the Bz component runs into noise for distances above 700m (1 kHz).
For higher frequency, noise already starts at 300m distance.

To the contrary, the Bx component is stable on the whole x axis.

We can also look how this looks expressed in amplitude and phase.

In [ ]:
self.showLineData(amphi=True, alim=[-3, 0], plim=[90, 180]);

Here we see that the Bx phases become noise at about 700m distance, the Bz phases already below 500. Note that there is no frequency dependence anymore, only for the Bz amplitude.

In [ ]:
# We choose a sounding about 500m from the transmitter
self.setPos(10, show=True)

In [ ]:
self.createDepthVector(rho=30)
print(self.depth)

In [ ]:
self.invertSounding(absError=noiseLevel, lam=100, cmp=[1, 0, 1], verbose=True);

In [ ]:
self.invertLine(absError=noiseLevel, verbose=False)

In [ ]:
self.showSection(cMin=5, cMax=500);